<a href="https://colab.research.google.com/github/mohammel-debug/G2P-Enlgish/blob/main/Final_project_morph_Darragh_Mohammed_Peio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **G2P FST Project: English IPA to Spanish Pronunciation Mapping**

The project was developed by Darragh Russell Kerins, Peio Delgado Legarza, and Mohammed El khamlichi.


# **Introduction**

This project models English pronunciation as produced by Spanish speakers using a rule-based finite-state transducer (FST) implemented in PyFoma. For example, the English word car (/kɑː/) is typically realised as [kɑr] by Spanish speakers, reflecting systematic phonological patterns driven by cross-linguistic transfer.
Rather than operating on graphemes directly which would introduce orthographic inconsistencies (e.g., ch realised as /tʃ/, /k/, or /ʃ/ in chart, character, and champagne) the system takes IPA-transcribed English words as input and applies phonological rewrite rules. Through FST composition, the transducer generates approximations of Spanish-accented English pronunciation.

The system's design is motivated by well-established structural differences between Spanish and English phonological inventories. Spanish has a small, stable five-vowel system and lacks several features central to English phonology, including schwa, vowel reduction, phonemic vowel length, and true diphthongal nuclei. These mismatches predict regular patterns of vowel neutralisation and diphthong simplification in learner English (Stockwell & Bowen, 1965; Finch, 1982).

Within an interlanguage framework (Selinker, 1972), Spanish-accented pronunciations are understood not as random errors, but as rule-governed outcomes shaped by first-language transfer and partial acquisition of the target system. Pedagogical descriptions of Spanish learners of English confirm the stability of these patterns across speakers and proficiency levels (Kenworthy, 1987). English vowel categories and reference transcriptions follow standard phonological descriptions (Roach, 2009).

This project demonstrates both the strengths and limitations of rule-based phonological modelling for non-native pronunciation, evaluating the extent to which deterministic FST rules can capture systematic patterns while identifying cases where lexical conditioning and phonological irregularity resist uniform treatment.

# **Dataset**

In [ ]:
!pip install pyfoma

In [ ]:
!wget -O wordlist_ipa.txt https://raw.githubusercontent.com/mohammel-debug/G2P-Enlgish/main/Wordlist%20with%20IPA%20transcripion

import pandas as pd

wordlist_df = pd.read_csv("wordlist_ipa.txt", sep=",")

lexicon = []
with open("wordlist_ipa.txt", encoding="utf-8") as f:
     next(f)

     for line in f:
         parts = line.strip().split(',')
         if len(parts) == 2:
            word = parts[0].strip()
            ipa = parts[1].strip().strip("/")
            lexicon.append((word, ipa))
print(lexicon)
print(len(wordlist_df))


--2026-01-18 17:09:44--  https://raw.githubusercontent.com/mohammel-debug/G2P-Enlgish/main/Wordlist%20with%20IPA%20transcripion
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2218 (2.2K) [text/plain]
Saving to: ‘wordlist_ipa.txt’

wordlist_ipa.txt    100%[===================>]   2.17K  --.-KB/s    in 0s      

2026-01-18 17:09:44 (37.9 MB/s) - ‘wordlist_ipa.txt’ saved [2218/2218]

[('that', 'ðæt'), ('this', 'ðɪs'), ('thought', 'θɔːt'), ('give', 'ɡɪv'), ('verb', 'vɜːb'), ('win', 'wɪn'), ('hello', 'həˈləʊ'), ('sing', 'sɪŋ'), ('you', 'juː'), ('lazy', 'ˈleɪzi'), ('pleasure', 'ˈpleʒə'), ('bag', 'bæɡ'), ('school', 'skuːl'), ('rabbit', 'ˈræbɪt'), ('go', 'ɡəʊ'), ('day', 'deɪ'), ('home', 'həʊm'), ('time', 'taɪm'), ('about', 'əˈbaʊt'), ('noise', 'nɔɪz'), ('hour', 'ˈa

# **FSTS**

In [ ]:
from pyfoma import FST

fsts = {}
original_lexicon = FST.from_strings(lexicon)
lexicon_for_composition = FST.from_strings(lexicon)

We have created two FSTs, original_lexicon and lexicon_for_composition, because once it is used for the composition, it no longer returns the original output(English IPA), as it gets mixed with the rules.

# **Rules**

Taking into account that the order of rules is crucial, as the first rule is applied first. The order we will be as follows: consonants, diphthongs, and vowels.

## **1. Consonants**

English consonants are relatively regular compared to vowels, but Spanish speakers systematically substitute several consonants when speaking English. These substitutions reflect structural differences between the two phonological systems, particularly the absence of certain voiced fricatives and the different realisation of liquids and glides in Spanish.


###**/v/ → [f]**

Spanish lacks phonemic /v/ and does not allow voiced obstruents word-finally. Word-final /v/ is therefore devoiced to [f].

Example: give — /ɡɪv/ → [ɡɪf]

Input: ɡɪv

Output: ɡɪf

###**/v/ → [b]**

Spanish lacks the voiced labiodental fricative /v/, which is consistently realised as the voiced bilabial stop [b] in all positions except word-finally. This neutralisation reflects the phonemic inventory of Spanish, where orthographic v and b represent the same underlying phoneme with allophonic variation between [b] and [β].

Example: verb — /vɜːb/ → [bɜːb]

In this context, both the initial /v/ and the medial /b/ are realised as [b], collapsing the English phonemic distinction.



###**/ɡ/ → [k]**

Spanish devoices all final stops. An English word-final /ɡ/ is therefore pronounced as [k].

Example: bag — /bæɡ/ → [bæk]

Input: bæɡ

Output: bæk



### **/h/ → [x]**

English /h/ is a voiceless glottal fricative with minimal oral constriction. Spanish speakers substitute the velar fricative [x] (equivalent to Spanish j), which has a more prominent place of articulation.

Example: hello — /həˈləʊ/ → [xəˈləʊ]

Input: həˈləʊ

Output: xələʊ


###**/ŋ/ → [n]**
The velar nasal /ŋ/ does not occur as an independent phoneme in Spanish, where it appears only as an allophone of /n/ before velar stops. Spanish speakers substitute the alveolar nasal [n] in all contexts.

Example: sing — /sɪŋ/ → [sɪn]

Input: sɪŋ

Output: sɪn

###**/j/ → [ʝ]**

English /j/ is realised as a palatal approximant, while Spanish speakers produce a voiced palatal fricative [ʝ] (equivalent to Spanish ll or y), which involves greater constriction.

Example: you — /juː/ → [ʝuː]

Input: juː

Output: ʝuː


###**/z/ → [s]**

Spanish lacks the voiced alveolar fricative /z/. All instances of English /z/ are devoiced to [s], reflecting the absence of voicing contrasts among Spanish fricatives.

Example: lazy — /ˈleɪzi/ → [ˈleɪsi]

Input: leɪzi

Output: leɪsi

###**/ʒ/ → [ʃ]**

The voiced postalveolar fricative /ʒ/ does not exist in Spanish. Spanish speakers substitute the voiceless counterpart [ʃ], neutralising the voicing distinction.

Example: pleasure — /ˈpleʒə/ → [ˈpleʃə]

Input: pleʒə

Output: pleʃə

###**/r/ → [r]**

The pronunciation of the /r/ in English and in Spanish is different. Even if the IPA symbol for the /r/ is the same in both languages, the pronunciation of all the ‘r’s in English by Spanish speakers is close to the pronunciation of the alveolar tap /r/ in Spanish, which is close to the pronuncioation of 'rr'. Even if there is no symbol in IPA to signal this, this is something that is taken into account here.

Example: rabbit — /ˈræbɪt/ → [ˈræbɪt]

Input: ræbɪt

Output: ræbɪt

###  **/s/ → [es]**

Spanish speakers tend to add an /e/ to the start of the English words that start with an /s/ and are followed by a consonant.

Example: school — /ˈskuːl' — / 'eskuːl'

Input: skuːl

Output: eskuːl



In [ ]:
fsts['rule_pre'] = FST.re("$^rewrite('ˈ':'' | 'ˌ':'')")


fsts['rule_v_end'] = FST.re("$^rewrite(v:f / _ #)")
fsts['rule_g_end'] = FST.re("$^rewrite(ɡ:k / _ #)")
fsts['rule_v'] = FST.re("$^rewrite(v:b)")
fsts['rule_h'] = FST.re("$^rewrite(h:x)")
fsts['rule_ŋ'] = FST.re("$^rewrite(ŋ:n)")
fsts['rule_j'] = FST.re("$^rewrite(j:ʝ /# _ )")
fsts['rule_z'] = FST.re("$^rewrite(z:s)")
fsts['rule_ʒ'] = FST.re("$^rewrite(ʒ:ʃ)")
fsts['rule_r'] = FST.re("$^rewrite(r:r)")


fsts['C'] = FST.re("b|c|d|f|g|h|j|k|l|m|n|p|q|r|s|t|v|w|x|y|z")
fsts['rule_s'] = FST.re("$^rewrite(s:(es) / # _ $C)", fsts)

fsts['rule_ð'] = FST.re("$^rewrite(ð:d)")
fsts['rule_θ'] = FST.re("$^rewrite(θ:t)")

consonants_grammar = FST.re("$rule_pre @ $rule_v_end @ $rule_g_end @ $rule_v @ "
                            "$rule_h @ $rule_ŋ @ $rule_j @ $rule_z @ "
                            "$rule_ʒ @ $rule_r @ $rule_s @ $rule_ð @ $rule_θ", fsts)

In [ ]:
consonants_fst = lexicon_for_composition @ consonants_grammar

In [ ]:
consonants_test = ['this', 'thought', 'give', 'verb', 'win', 'hello',
                  'sing', 'you', 'lazy', 'pleasure', 'bag',
                   'school', 'rabbit']

for word in consonants_test:
    original_ipa = list(original_lexicon.generate(word))
    transformed_ipa = list(consonants_fst.generate(word))
    print(f"Word: {word}")
    print(f"Original IPA: {original_ipa}")
    print(f"Transformed: {transformed_ipa}")
    print("\n")

Word: this
Original IPA: ['ðɪs']
Transformed: ['dɪs']


Word: thought
Original IPA: ['θɔːt']
Transformed: ['tɔːt']


Word: give
Original IPA: ['ɡɪv']
Transformed: ['ɡɪf']


Word: verb
Original IPA: ['vɜːb']
Transformed: ['bɜːb']


Word: win
Original IPA: ['wɪn']
Transformed: ['wɪn']


Word: hello
Original IPA: ['həˈləʊ']
Transformed: ['xələʊ']


Word: sing
Original IPA: ['sɪŋ']
Transformed: ['sɪn']


Word: you
Original IPA: ['juː']
Transformed: ['ʝuː']


Word: lazy
Original IPA: ['ˈleɪzi']
Transformed: ['leɪsi']


Word: pleasure
Original IPA: ['ˈpleʒə']
Transformed: ['pleʃə']


Word: bag
Original IPA: ['bæɡ']
Transformed: ['bæk']


Word: school
Original IPA: ['skuːl']
Transformed: ['eskuːl']


Word: rabbit
Original IPA: ['ˈræbɪt']
Transformed: ['ræbɪt']




## **2. Diphthongs**

English diphthongs do NOT exist as phonemic units in Spanish.

As documented in contrastive phonology (e.g. Roach 2009; Swan & Smith 2001)
English diphthongs do not exist in Spanish phonology, so Spanish speakers tend to realise them as **two separate vowels**.



To model this, each diphthong is handled using a **two-step transformation**:



1. Map the **first element** to its Spanish equivalent (with strict context)

2. Map the **second element** to its Spanish equivalent (with strict context)



**Context restrictions (`/ _ X` and `/ Y _`) are CRITICAL**   

They prevent these rules from incorrectly affecting other vowels in the word.

**Rule Composition**

- All diphthong rules must be **applied sequentially**

- **Order matters**:

  -  **diphthong rules are applied BEFORE standalone vowel rules**

  - This prevents rule bleeding and incorrect vowel transformations



###**/əʊ/ → [ou]  (GOAT vowel, RP)**

English schwa-based diphthongs are commonly "decentralised" by Spanish speakers, yielding a full mid-back vowel followed by [u].



**Example:** *go* — `/ɡəʊ/ → [ɡou]`



- **Step 1:** `ə → o / _ ʊ`

- **Step 2:** `ʊ → u / o _`




###**/eɪ/ → [ei]  (FACE vowel)**

Spanish has a stable [e] and [i] but no closing diphthong.

Learners typically realise /eɪ/ as a vowel sequence rather than a glide.





### `/eɪ/ → [ei]`

**Example:** *day* — `/deɪ/ → [dei]`



- **Step 1:** `e → e / _ ɪ`

- **Step 2:** `ɪ → i / e _`




### **/oʊ/ → [ou]  (Home vowel)**

As with /əʊ/, Spanish speakers favour a full back vowel sequence.



**Example:** *home* — `/hoʊm/ → [houm]`



- **Step 1:** `o → o / _ ʊ`

- **Step 2:** `ʊ → u / o _`



###**/aɪ/ → [ai]  (PRICE vowel)**

Spanish has /a/ and /i/ but no diphthongal nucleus with a moving target.



**Example:** *time* — `/taɪm/ → [taim]`



- **Step 1:** `a → a / _ ɪ`

- **Step 2:** `ɪ → i / a _`



### **/aʊ/ → [au]  (MOUTH vowel)**

English /aʊ/ is typically rendered as a sequence of full vowels in

 Spanish-accented English.



**Example:** *about* — `/əˈbaʊt/ → [əbaut]`



- **Step 1:** `a → a / _ ʊ`

- **Step 2:** `ʊ → u / a _`

###**/ɔɪ/ → [oi]  (CHOICE vowel)**

Spanish lacks mid–high closing diphthongs; /ɔɪ/ is typically flattened

into a stable vowel sequence.

**Example:** *noise* — `/nɔɪz/ → [nois]`



- **Step 1:** `ɔ → o / _ ɪ`

- **Step 2:** `ɪ → i / o _`



###**/ʊə/ → [uo]  (CURE vowel)**

Centring diphthongs are especially unstable for Spanish learners due to the absence of schwa and r-coloured nuclei.



**Example:** *hour* — `/ˈaʊə/ → [auo]`



- **Step 1:** `ʊ → u / _ ə`

- **Step 2:** `ə → o / u _`



### **/eə/ → [ea]  (SQUARE vowel)**

English centring diphthongs are commonly realised as vowel hiatus

rather than glide movement in Spanish-accented English



**Example:** *hair* — `/heə/ → [hea]`



- **Step 1:** `e → e / _ ə`

- **Step 2:** `ə → a / e _`



In [ ]:
fsts['rule_əʊ'] = FST.re("$^rewrite(ə:o / _ ʊ) @ $^rewrite(ʊ:u / o _)")
fsts['rule_eɪ'] = FST.re("$^rewrite(e:e / _ ɪ) @ $^rewrite(ɪ:i / e _)")
fsts['rule_oʊ'] = FST.re("$^rewrite(o:o / _ ʊ) @ $^rewrite(ʊ:u / o_)")
fsts['rule_aɪ'] = FST.re("$^rewrite(a:a / _ ɪ) @ $^rewrite(ɪ:i / a _)")
fsts['rule_aʊ'] = FST.re("$^rewrite(a:a / _ ʊ) @ $^rewrite(ʊ:u / a _)")
fsts['rule_ɔɪ'] = FST.re("$^rewrite(ɔ:o / _ ɪ) @ $^rewrite(ɪ:i / o _)")
fsts['rule_ʊə'] = FST.re("$^rewrite(ʊ:u / _ ə) @ $^rewrite(ə:o / u _)")
fsts['rule_eə'] = FST.re("$^rewrite(e:e / _ ə) @ $^rewrite(ə:a / e _)")


diphthongs_grammar = FST.re("$rule_əʊ @ $rule_eɪ @ $rule_oʊ @ $rule_aɪ @ "
                            "$rule_aʊ @ $rule_ɔɪ @ $rule_ʊə @ $rule_eə", fsts)

In [ ]:
diphthongs_fst = lexicon_for_composition @ diphthongs_grammar

In [ ]:
diphthongs_test = ['go', 'day', 'home', 'time', 'about', 'noise', 'hour', 'here']

for word in diphthongs_test:
    original_ipa = list(original_lexicon.generate(word))
    transformed_ipa = list(diphthongs_fst.generate(word))
    print(f"Word: {word}")
    print(f"Original IPA: {original_ipa}")
    print(f"Transformed: {transformed_ipa}")
    print("\n")

Word: go
Original IPA: ['ɡəʊ']
Transformed: ['ɡou']


Word: day
Original IPA: ['deɪ']
Transformed: ['dei']


Word: home
Original IPA: ['həʊm']
Transformed: ['xoum']


Word: time
Original IPA: ['taɪm']
Transformed: ['taim']


Word: about
Original IPA: ['əˈbaʊt']
Transformed: ['əbaut']


Word: noise
Original IPA: ['nɔɪz']
Transformed: ['nois']


Word: hour
Original IPA: ['ˈaʊə']
Transformed: ['auo']


Word: here
Original IPA: ['hɪə']
Transformed: ['xɪə']




## **3. Vowels**

English monophthong vowels do not map straightforwardly onto Spanish vowels due to inventory mismatches and differences in vowel quality. Spanish has a five-vowel system (/a e i o u/) with minimal allophonic variation, while English contrasts a larger set of vowel qualities and phonemic length distinctions.

Spanish speakers systematically neutralise English vowel contrasts, substituting the nearest available Spanish vowel. These patterns are well documented in contrastive phonology (Stockwell & Bowen 1965; Finch 1982; Kenworthy 1987). The rules below apply after diphthong resolution to preserve diphthong structure and prevent premature neutralisation.

**Rule Composition**

Monophthong rules apply after diphthong rules to preserve diphthong

*   Monophthong rules apply after diphthong rules to preserve diphthong structure.
*   All substitutions are context-free, reflecting broad phonemic merger rather than conditioned allophony.

*   Rule order prevents premature vowel neutralisation that would
otherwise bleed diphthong transformations.

###**/æ/ → [a]**

Spanish lacks the low front vowel /æ/, which is realised as [a].

Example: cat — /kæt/ → [kat]

###**/ʌ/ → [a]**

The mid-central vowel /ʌ/ is typically merged with [a].

Example: cup — /kʌp/ → [kap]

###**/ɒ/ → [o]**

The open back rounded vowel /ɒ/ is raised to [o].

Example: hot — /hɒt/ → [xot]

###**/uː/ → [u]**

Long vowels are shortened, as Spanish does not contrast vowel length.

Example: you — /juː/ → [ʝu]

###**/ɔː/ → [o]**

The long mid-back vowel /ɔː/ is neutralised to [o].

Example: thought — /θɔːt/ → [tot]

###**/ʊ/ → [u]**

The lax vowel /ʊ/ is replaced by tense [u], as Spanish lacks a lax–tense distinction.

Example: book — /bʊk/ → [buk]

### **/iː/ → [i]**

The close front unrounded vowel /iː/ does not exist in Spanish, so it is realised using the standard /i/ Spanish vowel.

Example: team — /tiːm/ → [tim]

In [ ]:

fsts['rule_æ'] = FST.re("$^rewrite(æ:a)")
fsts['rule_ʌ'] = FST.re("$^rewrite(ʌ:a)")
fsts['rule_ɒ'] = FST.re("$^rewrite(ɒ:o)")
fsts['rule_uː'] = FST.re("$^rewrite((uː):u)")
fsts['rule_ɔː'] = FST.re("$^rewrite((ɔː):o)")
fsts['rule_iː'] = FST.re("$^rewrite((iː):i)")
fsts['rule_ʊ'] = FST.re("$^rewrite(ʊ:u)")

vowels_grammar = FST.re("$rule_æ @ $rule_ʌ @ $rule_ɒ @"
                        "$rule_uː @ $rule_ɔː @ $rule_iː @ $rule_ʊ", fsts)


In [ ]:
vowels_fst = lexicon_for_composition @ vowels_grammar

In [ ]:
vowels_test = ['shit', 'cat', 'cup', 'hot', 'full', 'you', 'thought']

for word in vowels_test:
    original_ipa = list(original_lexicon.generate(word))
    transformed_ipa = list(vowels_fst.generate(word))
    print(f"Word: {word}")
    print(f"Original IPA: {original_ipa}")
    print(f"Transformed: {transformed_ipa}")
    print("\n")

Word: shit
Original IPA: ['ʃɪt']
Transformed: ['ʃɪt']


Word: cat
Original IPA: ['kæt']
Transformed: ['kat']


Word: cup
Original IPA: ['kʌp']
Transformed: ['kap']


Word: hot
Original IPA: ['hɒt']
Transformed: ['xot']


Word: full
Original IPA: ['fʊl']
Transformed: ['ful']


Word: you
Original IPA: ['juː']
Transformed: ['ʝu']


Word: thought
Original IPA: ['θɔːt']
Transformed: ['tot']




## **4. System Testing and Composition of All Rules**

Before proceeding with further refinements, the core grammar was tested to verify correct rule interaction and output consistency.

In [ ]:
fst_1 = lexicon_for_composition @ consonants_grammar @ diphthongs_grammar @ vowels_grammar

In [ ]:
test_sample =    ['that', 'thought', 'give', 'verb', 'win', 'hello',
                  'sing', 'you', 'lazy', 'pleasure', 'bag',
                   'school', 'rabbit' # conosnants

                   'go', 'day', 'home', 'time', 'about', 'noise', 'hour', 'here' # diphthongs

                    'sheet', 'cat', 'cup', 'hot', 'full', 'you', 'thought' #vowels
                   ]
results = []

for word in test_sample:
    try:
        ipa_list = list(original_lexicon.generate(word))
        ipa = ipa_list[0] if ipa_list else "N/A"

        transformed_list = list(fst_1.generate(word))
        transformed = transformed_list[0] if transformed_list else "N/A"

        results.append({'word': word, 'ipa': ipa, 'transformed': transformed})

    except Exception as e:
        continue

results_df = pd.DataFrame(results)

results_df.head(10)


,word,ipa,transformed
0,that,ðæt,dat
1,thought,θɔːt,tot
2,give,ɡɪv,ɡɪf
3,verb,vɜːb,bɜːb
4,win,wɪn,wɪn
5,hello,həˈləʊ,xəlou
6,sing,sɪŋ,sɪn
7,you,juː,ʝu
8,lazy,ˈleɪzi,leisi
9,pleasure,ˈpleʒə,pleʃə


As the basic rules now are created, the next step would be adding more sophisticated rules such as `schwa` and `rs`.

## **5. Schwa**

Schwa is particularly challenging, as it does not exist in Spanish and maps variably to different vowels even in identical phonological contexts. For example, words ending in /-ən/ such as happen (/ˈhæpən/), listen (/ˈlɪsən/), and even (/ˈiːvən/) are typically realised with [en], while orthographically similar words like button (/ˈbʌtən/), common (/ˈkɒmən/), and person (/ˈpɜːsən/) are often realised with [on]. This variability appears to be lexically conditioned rather than phonologically predictable, making it hard to apply a one-rule-fits-all approach.

The following rules attempt to model the most regular patterns while acknowledging these limitations.

**Rule Strategy**

Since Spanish speakers tend to substitute English vowels with their Spanish equivalents, and given that PyFoma applies only the first matching rule in a given context, a general approach is needed. The most frequent pattern involves words ending in /-ən/, which are typically realised with [en]. Therefore, the primary schwa rule targets this context:

Rule: ə → e / _ n #

This captures common cases such as happen → [hapen], listen → [lisen], and given → [giben], while acknowledging that less frequent patterns (e.g., /-ən/ → [on]) remain unmodeled due to the constraints of deterministic rule application.

In [ ]:
fsts['rule_ə_final_n'] = FST.re("$^rewrite(ə:e / _ n #)")  # happen, hapen

In [ ]:
grammar_schwa= FST.re("$rule_ə_final_n", fsts)

In [ ]:
fst_1 = lexicon_for_composition @ consonants_grammar @ diphthongs_grammar @ vowels_grammar @ grammar_schwa

In [ ]:
vowels_test = ['happen', 'given', 'human', 'common', 'listen', 'even', 'cotton',
               'little', 'middle', 'table', 'people', 'able', 'final']

for word in vowels_test:
    original_ipa = list(original_lexicon.generate(word))
    transformed_ipa = list(fst_1.generate(word))
    print(f"Word: {word}")
    print(f"Original IPA: {original_ipa}")
    print(f"Transformed: {transformed_ipa}")
    print("\n")

Word: happen
Original IPA: ['ˈhæpən']
Transformed: ['xapen']


Word: given
Original IPA: ['ˈɡɪvən']
Transformed: ['ɡɪben']


Word: human
Original IPA: ['ˈhjuːmən']
Transformed: ['xjumen']


Word: common
Original IPA: ['ˈkɒmən']
Transformed: ['komen']


Word: listen
Original IPA: ['ˈlɪsən']
Transformed: ['lɪsen']


Word: even
Original IPA: ['ˈiːvən']
Transformed: ['iben']


Word: cotton
Original IPA: ['ˈkɒtən']
Transformed: ['koten']


Word: little
Original IPA: ['ˈlɪtəl']
Transformed: ['lɪtəl']


Word: middle
Original IPA: ['ˈmɪdəl']
Transformed: ['mɪdəl']


Word: table
Original IPA: ['ˈteɪbəl']
Transformed: ['teibəl']


Word: people
Original IPA: ['ˈpiːpəl']
Transformed: ['pipəl']


Word: able
Original IPA: ['ˈeɪbəl']
Transformed: ['eibəl']


Word: final
Original IPA: ['ˈfaɪnəl']
Transformed: ['fainəl']




**Additional Rule: /-əl/ → [ol] after /b/**

Spanish speakers typically realise words like table and people as [teibol] and [piːpol], suggesting a context-specific pattern where schwa becomes [o] when preceded by /b/ and followed by /l/ word-finally.

Rule: ə → o / b _ l

Examples:

table — /ˈteɪbəl/ → [teibol]

able — /ˈeɪbəl/ → [eibol]

people — /ˈpiːpəl/ → [piːpol]

This rule applies in a more restricted context than the general /-ən/ rule, demonstrating how schwa realisation varies based on the surrounding phonological environment.

In [ ]:
fsts['rule_able'] = FST.re("$^rewrite(ə:o / b _ l #)")
list(fsts['rule_able'].generate("ˈeɪbəl"))

['ˈeɪbol']

In [ ]:
grammar_schwa= FST.re("$rule_ə_final_n @ $rule_able", fsts)

In [ ]:
list(grammar_schwa.generate("ˈeɪbəl"))

['ˈeɪbol']

In [ ]:
fst_1 = lexicon_for_composition @ consonants_grammar @ diphthongs_grammar @ vowels_grammar @ grammar_schwa

In [ ]:
list(fst_1.generate("able"))

['eibol']

## **6. "R" Insertion and Long Vowels**

The relationship between English long vowels and Spanish /r/ seems to be challenging as Spanish speakers often add /r/ when they see the letter "r" in spelling, converting English /ɑː/ to Spanish /ar/. However, this creates errors. For example, "father", /ˈfɑːðə/, would become /fɑrder/ instead of /fɑder/ and words like "calm", /kɑːm/, show that long vowels can arise from silent letters rather than "r" influence. Thus, it is really challenging to decide when to insert /r/.

One approach is to find a specific frequent context. For instance, adding a final /r/ to English long vowels at word endings. Each rule targets a specific vowel.

**/ɑː/ → [ɑr]**

/kɑː/ becomes /kar/

**/ɔː/ → [ɔr]**

/'dɔː/ becomes [dɔr]

**/ɜː/ → /ɜr/**

/hɜː/ becomes [xɜr]

**/ə/ → /ər/**

The final vowel in /ˈfɑːðə/ becomes [fɑːdər]. This systematic /r/ insertion approximates how Spanish speakers often pronounce English words that are spelled with a final "r".

In [ ]:
fsts['rule_insert_r_final_ɑː'] = FST.re("$^rewrite((ɑː):(ɑr) / _ #)")
fsts['rule_insert_r_final_ɔː'] = FST.re("$^rewrite((ɔː):(ɔr) / _ #)")
fsts['rule_insert_r_final_ɜː'] = FST.re("$^rewrite((ɜː):(ɜr) / _ #)")
fsts['rule_insert_r_final_ə'] = FST.re("$^rewrite(ə:(ər) / _ #)")


grammar_r_insertion= FST.re(
"$rule_insert_r_final_ɑː @ $rule_insert_r_final_ɔː @ $rule_insert_r_final_ɜː @  $rule_insert_r_final_ə ", fsts)

In [ ]:
fst_1 = lexicon_for_composition @ consonants_grammar @ diphthongs_grammar @ grammar_r_insertion @ vowels_grammar @ grammar_schwa

In [ ]:
vowels_test = ['father', 'teacher', 'colour', 'car', 'clearer', 'greater', 'bigger',
               'preacher', 'door', 'more', 'her', 'sir']

for word in vowels_test:
    original_ipa = list(original_lexicon.generate(word))
    transformed_ipa = list(fst_1.generate(word))
    print(f"Word: {word}")
    print(f"Original IPA: {original_ipa}")
    print(f"Transformed: {transformed_ipa}")
    print("\n")

Word: father
Original IPA: ['ˈfɑːðə']
Transformed: ['fɑːdər']


Word: teacher
Original IPA: ['ˈtiːtʃə']
Transformed: ['titʃər']


Word: colour
Original IPA: ['ˈkʌlə']
Transformed: ['kalər']


Word: car
Original IPA: ['kɑː']
Transformed: ['kɑr']


Word: clearer
Original IPA: ['ˈklɪərə']
Transformed: ['klɪərər']


Word: greater
Original IPA: ['ˈɡreɪtə']
Transformed: ['ɡreitər']


Word: bigger
Original IPA: ['ˈbɪɡə']
Transformed: ['bɪɡər']


Word: preacher
Original IPA: ['ˈpriːtʃə']
Transformed: ['pritʃər']


Word: door
Original IPA: ['dɔː']
Transformed: ['do']


Word: more
Original IPA: ['mɔː']
Transformed: ['mo']


Word: her
Original IPA: ['hɜː']
Transformed: ['xɜr']


Word: sir
Original IPA: ['sɜː']
Transformed: ['sɜr']




Since in diphthongs rules the long vowel /ɔː/ is replaced by a short vowel /o/, the rule of inserting /r/ in the final of the long vowel /ɔː/ failed ("more", /'mɔː'/,['mo']).


## **7. Evaluation**

In [ ]:
test_words = [
    'start', 'cave', 'dive', 'dove', 'jog', 'hug', 'violence', 'vision',
    'view', 'visit', 'hard', 'habit', 'hack', 'string', 'sitting',
    'disappearing', 'yard', 'yell', 'yes', 'amazed', 'blaze', 'haze',
    'decision', 'bath', 'cloth', 'thrive', 'feather', 'leather', 'vote',
    'van', 'wave', 'wheel', 'whale', 'jungle', 'king', 'young', 'measure',
    'zip', 'buzz', 'rose', 'show', 'fish', 'chair', 'watch', 'page',
    'judge', 'boot', 'food', 'book', 'good', 'bat', 'man', 'luck', 'son',
    'pot', 'fox', 'pool', 'brown', 'town', 'boy', 'toy', 'beer', 'deer',
    'air', 'hair', 'sure', 'tour', 'butter', 'paper', 'mother', 'sister',
    'water', 'better', 'letter', 'doctor', 'actor', 'finger', 'anger',
    'summer', 'winner', 'player', 'higher', 'flower', 'power'
]

In [ ]:
all_results = []

for word in test_words:
    original_ipa = list(original_lexicon.generate(word))
    transformed_ipa = list(fst_1.generate(word))

    original_ipa_str = original_ipa[0] if original_ipa else "N/A"
    transformed_ipa_str = transformed_ipa[0] if transformed_ipa else "N/A"

    all_results.append({
        'Word': word,
        'Original IPA': original_ipa_str,
        'Transformed IPA': transformed_ipa_str
    })

results_df = pd.DataFrame(all_results)
print(results_df.to_markdown(index=False))

| Word         | Original IPA   | Transformed IPA   |
|:-------------|:---------------|:------------------|
| start        | stɑːt          | estɑːt            |
| cave         | keɪv           | keif              |
| dive         | daɪv           | daif              |
| dove         | dʌv            | daf               |
| jog          | dʒɒɡ           | dʃok              |
| hug          | hʌɡ            | xak               |
| violence     | ˈvaɪələns      | baiələns          |
| vision       | ˈvɪʒən         | bɪʃen             |
| view         | vjuː           | bju               |
| visit        | ˈvɪzɪt         | bɪsɪt             |
| hard         | hɑːd           | xɑːd              |
| habit        | ˈhæbɪt         | xabɪt             |
| hack         | hæk            | xak               |
| string       | strɪŋ          | estrɪn            |
| sitting      | ˈsɪtɪŋ         | sɪtɪn             |
| disappearing | ˌdɪsəˈpɪərɪŋ   | dɪsəpɪərɪn        |
| yard         | jɑːd       

# **Conclusion**

This project implemented a finite-state transducer approach to modeling Spanish-accented English. Some aspects of this accent are relatively regular; for example, Spanish speakers often insert an initial /e/ before words that start with 's' followed by a consonant, so “snail” may be pronounced /esneɪl/. This reflects Spanish syllable structure and applies consistently across many words.

However, consonant pronunciation is not consistently predictable. The rule transducing /z/ to /s/ illustrates this: “zeal” follows the rule and is pronounced /siːl/, while “zebra” may be pronounced /θiːbrə/, showing exceptions that highlight English’s irregularity and the influence of orthography and lexical familiarity.

The insertion of /r/ is another challenging feature. Spanish speakers typically realise all /r/ sounds as tapped or trilled, which complicates IPA-based transduction. For instance, “player” (/pleɪə/) becomes /pleɪər/ with both a vowel and /r/ added. While appending -er to schwa-final words works in some cases, it does not generalise: words ending in -or or -ar, such as “professor” or “burglar,” are not pronounced with /r/ in British English. Long vowels are also affected: /ɜː/ in “bird” becomes /er/, while in “work” it becomes /or/, reflecting Spanish speakers’ reliance on spelling-to-sound correspondences. In Hiberno-English, however, adding the /r/ in these words is considered normal and part of standard pronunciation, showing that such pronunciations depend on regional accent variation and the subjectivity of what is considered the “correct” form.

These challenges illustrate the broader limitations of finite-state modelling for interlanguage phenomena. English phonological opacity (Chomsky & Halle, 1968), combined with L1 transfer and orthographic interference, produces patterns that deterministic FST rules cannot systematically capture. Additionally, Foma’s restriction that only one rule may apply per symbol can create conflicts, reducing coverage.

In conclusion, while FSTs handle regular phonological patterns well, they struggle with semi-regular, lexically conditioned, and orthographically influenced features of Spanish-accented English. Probabilistic approaches with access to orthography and lexical exceptions would provide a more robust model than classical finite-state architectures.

## **Neural G2P Summary and Conclusion**

To address the limitations of rule-based phonological modelling, a preliminary neural IPA-to-IPA sequence-to-sequence model was explored. English IPA was treated as the input sequence and Spanish-influenced IPA, generated by a finite-state transducer (FST), as the target output. Due to the restricted symbol coverage of the FST, the model was trained on a small dataset of 137 IPA pairs, making this a low-resource experiment.

Despite this data sparsity, the Transformer showed clear learning over 150 epochs, with validation loss decreasing to approximately 2.05. Qualitative evaluation indicates that the model successfully internalised several core accent patterns, including dental fricative substitution (/ð, θ/ → /d, t/) and vowel tensing (/ɪ/ → /i/).

Generalisation was further tested on four unseen words (tower, vest, snake, and brave) using manually supplied IPA. The model produced plausible accented forms that preserved syllable structure and major segmental patterns. For example, tower reflected correct diphthong handling, vest showed voicing patterns consistent with Spanish-accented English, and consonant clusters in snake and brave were largely maintained. While some outputs exhibited segment deletion or vowel instability, these errors are consistent with low-resource neural training rather than model failure. Overall, the results demonstrate that neural models can extract and apply phonological regularities even from limited, rule-derived supervision.

#**Future Work: A Hybrid Neural–FST Approach**

The main weakness of the current system is the limited coverage of the rule-based FST, which drastically reduces the amount of usable training data. Future work should adopt a hybrid approach that combines the strengths of rule-based and neural methods. In particular, the FST could be used to generate a seed dataset of Spanish-accented IPA forms from a moderate corpus of English words. This dataset could then be scaled up using an LLM, which would generate multiple plausible accent variants for each word, preserving syllable structure and accent-specific phonological patterns.

Training a neural model on this enriched, synthetically augmented dataset would allow it to move beyond rigid rule application and better capture subtle, word-specific pronunciation patterns. This hybrid framework retains the interpretability of finite-state phonology while leveraging the flexibility and generalisation capacity of neural models, offering a more scalable solution for accented pronunciation modelling.

## Neural G2P and  FST dataset generation

In [ ]:
# Step 1: Install pyfoma (if not already installed)
!pip install pyfoma

# Step 2: Download the EZTransformer module
!wget https://raw.githubusercontent.com/mhulden/eztransformer/refs/heads/main/eztr.py

# Step 3: Load the lexicon data
!wget -O wordlist_ipa.txt https://raw.githubusercontent.com/mohammel-debug/G2P-Enlgish/main/Wordlist%20with%20IPA%20transcripion

import pandas as pd

wordlist_df = pd.read_csv("wordlist_ipa.txt", sep=",")

lexicon = []
with open("wordlist_ipa.txt", encoding="utf-8") as f:
    next(f)
    for line in f:
        parts = line.strip().split(',')
        if len(parts) == 2:
            word = parts[0].strip()
            ipa = parts[1].strip().strip("/")
            lexicon.append((word, ipa))

print(f"Loaded {len(lexicon)} word-IPA pairs")
print("Sample entries:", lexicon[:3])

# Step 4: Create the FSTs
# Make sure you've run all the FST creation code before this step
# This includes: consonants_grammar, diphthongs_grammar, vowels_grammar, grammar_schwa, grammar_r_insertion

from pyfoma import FST

# Recreate the FSTs if needed
fsts = {}
original_lexicon = FST.from_strings(lexicon)
lexicon_for_composition = FST.from_strings(lexicon)

# Then define all your rules and create fst_1
# fst_1 = lexicon_for_composition @ consonants_grammar @ diphthongs_grammar @ vowels_grammar @ grammar_schwa @ grammar_r_insertion

# Step 5: Now create the neural training data
from eztr import EZTransformer
import pickle
from random import shuffle

neural_train_data = []
for word, ipa in lexicon:
    # Get the Spanish-transformed IPA from our FST
    transformed = list(fst_1.generate(word))
    if transformed:
        # Format: space-separated input and output
        input_seq = ' '.join(list(ipa))
        output_seq = ' '.join(list(transformed[0]))
        neural_train_data.append((input_seq, output_seq))

print(f"\nCreated {len(neural_train_data)} training examples")

# Step 6: Split into train/validation (90/10)
shuffle(neural_train_data)
split_idx = int(len(neural_train_data) * 0.9)
train_data = neural_train_data[:split_idx]
valid_data = neural_train_data[split_idx:]

print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(valid_data)}")

# Show example format
print("\nExample training data format:")
if train_data:
    print(f"Input:  {train_data[0][0]}")
    print(f"Output: {train_data[0][1]}")

# Step 7: Initialize and train the model
trf = EZTransformer(device='cuda')  # Use 'cpu' if no GPU available
trf.fit(train_data, valid_data=valid_data, print_validation_examples=2, max_epochs=150)

# Step 8: Test predictions
# Manually providing IPA for words NOT in the lexicon to test generalization
test_data = [
    ('tower', 't a ʊ ə'),
    ('vest',  'v e s t'),
    ('snake', 's n e ɪ k'),
    ('brave', 'b r e ɪ v')
]

print("\n=== Testing Generalization on Unseen Words ===")
for word, ipa in test_data:
    # Predict using the manual IPA string (must be space-separated)
    prediction = trf.predict([ipa])

    print(f"\nWord:      {word}")
    print(f"Input IPA: {ipa}")
    print(f"Predicted: {prediction[0]}")

    # Logic Check:
    # 'tower' should look like 't a u o' (following the pattern of 'power')
    # 'vest'  should look like 'b e s t' (following the pattern of 'visit')

--2026-01-18 17:09:53--  https://raw.githubusercontent.com/mhulden/eztransformer/refs/heads/main/eztr.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17140 (17K) [text/plain]
Saving to: ‘eztr.py.1’

eztr.py.1           100%[===================>]  16.74K  --.-KB/s    in 0.001s  

2026-01-18 17:09:53 (22.0 MB/s) - ‘eztr.py.1’ saved [17140/17140]

--2026-01-18 17:09:53--  https://raw.githubusercontent.com/mohammel-debug/G2P-Enlgish/main/Wordlist%20with%20IPA%20transcripion
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2218 (2.2K

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
Epoch 1/150: 100%|██████████| 1/1 [00:01<00:00,  1.28s/it]


Epoch 1: Training Loss: 3.908447
Epoch 1: Validation Loss: 4.125947
Model saved to best_model.pt

Validation Examples:
Input:     ˈ ɡ ɪ v ə n
Target:    ɡ ɪ b e n
Predicted: 

Input:     d ʌ v
Target:    d a f
Predicted: 



Epoch 2/150: 100%|██████████| 1/1 [00:00<00:00, 15.20it/s]

Epoch 2: Training Loss: 3.679591
Epoch 2: Validation Loss: 3.565251


...

Epoch 148: Training Loss: 1.037986
Epoch 148: Validation Loss: 2.476772

Validation Examples:
Input:     d ʌ v
Target:    d a f
Predicted: ɡ a d ɡ ə r

Input:     s t ɑ ː t
Target:    e s t ɑ ː t
Predicted: b o t ə r



Epoch 149/150: 100%|██████████| 1/1 [00:00<00:00, 21.03it/s]


Epoch 149: Training Loss: 1.017877
Epoch 149: Validation Loss: 2.520407

Validation Examples:
Input:     ˈ l ɪ s ə n
Target:    l ɪ s e n
Predicted: l ɪ s ɪ n

Input:     d e ɪ
Target:    d e i
Predicted: w e i d ə r



Epoch 150/150: 100%|██████████| 1/1 [00:00<00:00, 21.69it/s]


Epoch 150: Training Loss: 1.051760
Epoch 150: Validation Loss: 2.587010

Validation Examples:
Input:     d e ɪ
Target:    d e i
Predicted: w e i d ʃ

Input:     ˈ p l e ɪ ə
Target:    p l e i ə r
Predicted: p e i d ə r


=== Testing Generalization on Unseen Words ===

Word:      tower
Input IPA: t a ʊ ə
Predicted: t a u o

Word:      vest
Input IPA: v e s t
Predicted: b e i b o l

Word:      snake
Input IPA: s n e ɪ k
Predicted: w e i f

Word:      brave
Input IPA: b r e ɪ v
Predicted: b e i f


##FST generated Dataset

In [ ]:
import pandas as pd

# Create a DataFrame to view the training set
df_train = pd.DataFrame(train_data, columns=['English_IPA', 'Spanish_Target'])

# To see the whole set in Google Colab/Jupyter:
# df_train

# Or just the first 50 rows:
df_train.head(120)

,English_IPA,Spanish_Target
0,s ɪ ŋ,s ɪ n
1,b ɑ ː θ,b ɑ ː t
2,w ɒ t ʃ,w o t ʃ
3,m æ n,m a n
4,ˈ v ɪ ʒ ə n,b ɪ ʃ e n
...,...,...
115,k ɪ ŋ,k ɪ n
116,ˈ f ɑ ː ð ə,f ɑ ː d ə r
117,r ə ʊ z,r o u s
118,f ɒ k s,f o k s


#Bibliography

Chomsky, N., & Halle, M. (1968). The Sound Pattern of English. Harper & Row.

Finch, G. (1982). Contrastive Analysis of Spanish and English Phonology.

Kenworthy, J. (1987). Teaching English Pronunciation. Longman.
Roach, P. (2009). English Phonetics and Phonology (4th ed.). Cambridge University Press.

Selinker, L. (1972). "Interlanguage." International Review of Applied Linguistics, 10, 209–231.

Stockwell, R. P., & Bowen, J. D. (1965). The Sounds of English and Spanish. University of Chicago Press.